In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9.post4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the tokenizer

def formatting_prompts_func(examples):
    company_databases = examples["sql_context"]
    prompts = examples["sql_prompt"]
    sqls = examples["sql"]
    explanations = examples["sql_explanation"]

    merged_data = {"role_content": []}  # Creating a new field for role and content

    for company_database, prompt, sql, explanation in zip(company_databases, prompts, sqls, explanations):
        user_text = f"Below is an instruction that describes a task, paired with an input prompt. ### Instruction: Company database: {company_database} ### Input: SQL Prompt: {prompt}"
        assistant_text = f"SQL: {sql} Explanation: {explanation}"

        # Creating a single structure with user first, followed by assistant
        role_content_pair = [
            {
                "role": "user",
                "content": user_text
            },
            {
                "role": "assistant",
                "content": assistant_text
            }
        ]

        # Appending the pair to the merged data
        merged_data["role_content"].append(role_content_pair)

    return merged_data

# Loading the dataset
dataset = load_dataset("gretelai/synthetic_text_to_sql", split="train")

merged_data = dataset.map(formatting_prompts_func, batched=True, remove_columns=dataset.column_names)

# Printing the first entry to verify the structure
print(merged_data["role_content"][0])



README.md:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

(…)nthetic_text_to_sql_train.snappy.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

(…)ynthetic_text_to_sql_test.snappy.parquet:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

[{'content': "Below is an instruction that describes a task, paired with an input prompt. ### Instruction: Company database: CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01'); ### Input: SQL Prompt: What is the total volume of timber sold by each salesperson, sorted by salesperson?", 'role': 'user'}, {'content': 'SQL: SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_id = salesperson.salesperson_id GROUP BY salesperson_id, name ORDER BY total_volume DESC; Explanation: Joins timber_sales and salesperson tables, groups sales by salesperson,

In [ ]:
START_HEADER_ID = "<|start_header_id|>"
END_HEADER_ID = "<|end_header_id|>"
EOT_ID = "<|eot_id|>"
BEGIN_OF_TEXT = "<|begin_of_text|>"
SYSTEM_TEXT = "You are an assistant for SQL generation."  # Customize as needed

def formatting_prompts_func1(examples):
    role_contents = examples["role_content"]
    formatted_texts = []

    for role_content in role_contents:
        convo = ""

        # Adding system part at the beginning
        convo += f"{BEGIN_OF_TEXT}{START_HEADER_ID}system{END_HEADER_ID}\n\n{SYSTEM_TEXT}{EOT_ID}\n"

        if isinstance(role_content, list) and len(role_content) == 2:
            user_entry = role_content[0]  # User content
            assistant_entry = role_content[1]  # Assistant content

            user_role = user_entry.get("role", "user")
            user_content = user_entry.get("content", "")
            convo += f"{START_HEADER_ID}{user_role}{END_HEADER_ID}\n\n{user_content}{EOT_ID}\n"

            assistant_role = assistant_entry.get("role", "assistant")
            assistant_content = assistant_entry.get("content", "")
            convo += f"{START_HEADER_ID}{assistant_role}{END_HEADER_ID}\n\n{assistant_content}{EOT_ID}\n"

        formatted_texts.append(convo.strip())  # to remove any trailing whitespace

    return {"text": formatted_texts}

formatted_dataset = merged_data.map(formatting_prompts_func1, batched=True)

for i in range(3):  # Print first three samples for verification
    print(f"Sample {i+1}:\n{formatted_dataset['text'][i]}\n")


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Sample 1:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an assistant for SQL generation.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Below is an instruction that describes a task, paired with an input prompt. ### Instruction: Company database: CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01'); ### Input: SQL Prompt: What is the total volume of timber sold by each salesperson, sorted by salesperson?<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

SQL: SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson

In [ ]:
print(merged_data["role_content"][3])


[{'content': 'Below is an instruction that describes a task, paired with an input prompt. ### Instruction: Company database: CREATE TABLE trade_history (id INT, trader_id INT, stock VARCHAR(255), price DECIMAL(5,2), quantity INT, trade_time TIMESTAMP); ### Input: SQL Prompt: What is the total trade value and average price for each trader and stock in the trade_history table?', 'role': 'user'}, {'content': 'SQL: SELECT trader_id, stock, SUM(price * quantity) as total_trade_value, AVG(price) as avg_price FROM trade_history GROUP BY trader_id, stock; Explanation: This query calculates the total trade value and average price for each trader and stock by grouping the trade_history table by the trader_id and stock columns and summing up the product of price and quantity for each trade and getting the average price for each group.', 'role': 'assistant'}]


In [ ]:
print(formatted_dataset['text'][0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an assistant for SQL generation.<|eot_id|>
<|start_header_id|>user<|end_header_id|>

Below is an instruction that describes a task, paired with an input prompt. ### Instruction: Company database: CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01'); ### Input: SQL Prompt: What is the total volume of timber sold by each salesperson, sorted by salesperson?<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

SQL: SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_id = sale

In [ ]:
formatted_dataset[0]["text"]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are an assistant for SQL generation.<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\nBelow is an instruction that describes a task, paired with an input prompt. ### Instruction: Company database: CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01'); ### Input: SQL Prompt: What is the total volume of timber sold by each salesperson, sorted by salesperson?<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n\nSQL: SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_

In [ ]:
###Trainer

###Trainer

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
input_ids = tokenizer("SELECT * FROM users;", return_tensors="pt").input_ids
decoded = tokenizer.decode(input_ids[0])
print(decoded) #you can see it matches the query so the tokenizer is working fine


<|begin_of_text|>SELECT * FROM users;


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
3.658 GB of memory reserved.


###Training

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,1.167300
2,1.161700
3,1.169100
4,1.117900
5,0.786600
6,0.784300
7,0.657300
8,0.564200
9,0.523200
10,0.575600


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

218.8025 seconds used for training.
3.65 minutes used for training.
Peak reserved memory = 3.658 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 24.803 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Running the model You can change the instruction and input



In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "Write an SQL query to fetch the name and email of users who signed up in the last 30 days."},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWrite an SQL query to fetch the name and email of users who signed up in the last 30 days.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSQL: SELECT user_name, user_email FROM signups WHERE sign_up_date >= DATEADD(day, -30, GETDATE()) Explanation: This SQL query fetches the names and emails of users who signed up in the last 30 days.<|eot_id|>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting.

In [ ]:
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "Write an SQL query to fetch the name and email of users who signed up in the last 30 days."},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

SQL: SELECT user_name, email FROM signups WHERE sign_date >= DATEADD(day, -30, GETDATE()); Explanation: This SQL query fetches the names and emails of users who signed up in the last 30 days by comparing the current date to the date 30 days ago.<|eot_id|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

This ONLY saves the LoRA adapters, and not the full model.

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')